In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
import networkx as nx
from rt_bundled_ego_chord_diagram import RTBundledEgoChordDiagram
import rtsvg
rt = rtsvg.RACETrack()
edges_filename  = '../../data/stanford/facebook/1684.edges'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = _split_[0], _split_[1]
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df = pl.DataFrame(_lu_)
# Only Keep The Largest Component
g = rt.createNetworkXGraph(df, [('fm','to')])
comp_largest = None
for _comp_ in nx.connected_components(g):
    if comp_largest is None:            comp_largest = _comp_
    if len(_comp_) > len(comp_largest): comp_largest = _comp_
df = df.filter(pl.col('fm').is_in(comp_largest) & pl.col('to').is_in(comp_largest))
df.shape

In [ ]:
becd = RTBundledEgoChordDiagram(rt, df, [('fm','to')])
becd

In [ ]:
_lu_ = {'fm':[], 'to':[], 'src':[], 'dst':[], 'ct':[], 'co':[]}
_lu_['fm'].append('a'),  _lu_['to'].append('b'),  _lu_['src'].append(None), _lu_['dst'].append(None), _lu_['ct'].append(10), _lu_['co'].append('red')
_lu_['fm'].append('b'),  _lu_['to'].append('c'),  _lu_['src'].append(None), _lu_['dst'].append(None), _lu_['ct'].append(20), _lu_['co'].append('red')
_lu_['fm'].append('a'),  _lu_['to'].append('c'),  _lu_['src'].append(None), _lu_['dst'].append(None), _lu_['ct'].append(30), _lu_['co'].append('green')
_lu_['fm'].append(None), _lu_['to'].append(None), _lu_['src'].append(1),    _lu_['dst'].append(2),    _lu_['ct'].append(1),  _lu_['co'].append('blue')
_lu_['fm'].append(None), _lu_['to'].append(None), _lu_['src'].append(2),    _lu_['dst'].append(3),    _lu_['ct'].append(2),  _lu_['co'].append('blue')
_lu_['fm'].append(None), _lu_['to'].append(None), _lu_['src'].append(3),    _lu_['dst'].append(1),    _lu_['ct'].append(3),  _lu_['co'].append('orange')
_lu_['fm'].append(None), _lu_['to'].append(None), _lu_['src'].append(3),    _lu_['dst'].append(1),    _lu_['ct'].append(5),  _lu_['co'].append('purple')

df_test = pl.DataFrame(_lu_)
_df0_    = df_test.rename({'fm':'__fm__',  'to':'__to__',  'ct':'__count__', 'co':'__color__'}).filter(pl.col('__fm__').is_not_null() & pl.col('__to__').is_not_null())
_df0_    = _df0_.drop(set(_df0_.columns) - set(['__fm__','__to__','__count__','__color__']))

_df1_    = df_test.rename({'src':'__fm__', 'dst':'__to__', 'ct':'__count__', 'co':'__color__'}).filter(pl.col('__fm__').is_not_null() & pl.col('__to__').is_not_null())
_df1_    = _df1_.drop(set(_df1_.columns) - set(['__fm__','__to__','__count__','__color__']))
_df1_    = _df1_.with_columns(pl.col('__fm__').cast(pl.String), pl.col('__to__').cast(pl.String))

pl.concat([_df0_, _df1_])

In [ ]:
_tiles_ = []
for i in range(8):
    g   = rt.createNetworkXGraph(becd.df_communities, [('__fm__','__to__')])
    pos = nx.spring_layout(g)
    _tiles_.append(rt.link(becd.df_communities, [('__fm__','__to__')], pos, node_size='small', w=192, h=192))
rt.tile(_tiles_, spacer=2)

In [ ]:
g   = rt.createNetworkXGraph(df, [('fm','to')])
pos = nx.spring_layout(g)
rt.link(df, [('fm','to')], pos, w=512, h=512)